**BERT**

In [37]:
from transformers import BertTokenizer, BertForSequenceClassification
from torch.nn.functional import softmax
import torch
import pickle
import pandas as pd
from tqdm.notebook import tqdm

# Load local modules
from module_text_blocks import split_text_into_blocks, clean_text_blocks

In [15]:
# Load pdf text and headings from the pickle file
pdf_texts = pickle.load(open("pdf_texts.pkl", "rb"))
pdf_headings = pickle.load(open("pdf_headings.pkl", "rb"))

In [26]:
list(pdf_headings.items())[0:5]

[('COMBINED-Q4-17-Shareholder-Letter-FINAL',
  ['Q4 Results',
   'Forecast',
   'Content',
   'Product and Partnerships',
   'Competition',
   'Free Cash Flow and Capital Structure',
   'Board of Directors',
   'Summary',
   '22, 2018 Earnings Interview, 3pm PST']),
 ('FINAL-Q1-18-Shareholder-Letter',
  ['Q1 Results and Q2 Forecast',
   'Content',
   'Marketing',
   'Product and Partnerships',
   'Free Cash Flow and Capital Structure',
   'Board of Directors',
   'Reference',
   'Earnings Video Interview, 3pm PST']),
 ('FINAL-Q1-19-Shareholder-Letter',
  ['Q1 Results and Q2 Forecast',
   'Content',
   'Product and Partnerships',
   'Competition',
   'Cash Flow and Capital Structure',
   'Reference',
   'Appendix April 16, 2019 Earnings Interview, 3pm PST']),
 ('FINAL-Q1-20-Shareholder-Letter',
  ['(€1',
   'Q1 Results and Q2 Forecast',
   'Content',
   'Product',
   'Cash Flow and Capital Structure',
   'Reference']),
 ('FINAL-Q1-21-Shareholder-Letter',
  ['Q1 Results and Q2 Forecast',

In [16]:
# Load the pre-trained BERT model and tokenizer
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [29]:
sentiment_labels = {}
positive_sentiment_probs = {}
negative_sentiment_probs = {}

for pdf_name in tqdm(pdf_texts):
    # Input text containing Q4 financial results
    # input_text = "We had a beautiful Q4, completing a great year as internet TV expands globally. In 2017, we grew streaming revenue 36% to over $11 billion, added 24 million new memberships (compared to 19 million in 2016), achieved for the first time a full-year positive international contribution profit, and more than doubled global operating income. Average paid streaming memberships rose 25% year over year in Q4. Combined with a 9% increase in ASP, global streaming revenue growth amounted to 35%. Operating income of $245 million (7.5% margin) vs. $154 million prior year (6.2% margin) was slightly above our $238 million forecast. Operating margin for FY17 was 7.2%, on target with our goal at the beginning of this year. EPS was $0.41 vs. $0.15 last year and met our forecast of $0.41. There were several below the line items that affected net income, including a pre-tax $26 million non-cash unrealized loss from F/X remeasurement on our Eurobond. Our tax rate was helped by a $66 million foreign tax benefit, which partially offset a revaluation of our deferred tax assets and the impact from the mandatory deemed repatriation of accumulated foreign earnings related to the recent US tax reform. In Q4, we registered global net adds of 8.3 million, the highest quarter in our history and up 18% vs. last year’s record 7.05 million net adds. This exceeded our 6.3m forecast due primarily to stronger than expected acquisition fueled by our original content slate and the ongoing global adoption of internet entertainment. Geographically, the outperformance vs. guidance was broad-based. In the US, memberships rose by 2.0 million (vs. forecast of 1.25m) bringing total FY17 net adds to 5.3 million. ASP rose 5% year-over-year. Domestic contribution profit increased 5% year-over-year although contribution margin of 34.4% declined both on a year-over-year and sequential basis due to the marketing spend we noted in last quarter’s investor letter. Internationally, we added 6.36 million memberships (compared with guidance of 5.05m), a new record for quarterly net adds for this segment. Excluding a F/X impact of +$43 million, international revenue and ASP grew 59% and 12% year over year, respectively. The increase in ASP reflects price adjustments in a wide variety of our markets over the course of 2017. With contribution profit of $227 million in 2017 (4.5% contribution margin), the international segment delivered its first full year of positive contribution profit in our history. We took a $39m non-cash charge in Q4 for unreleased content we’ve decided not to move forward with. This charge was recognized in content expense in cost of revenues. Despite this unexpected expense, we slightly exceeded our contribution profit and operating income forecast due to our stronger than expected member growth and the timing of international content spend."

    # pdf_name = "FINAL-Q1-19-Shareholder-Letter"

    text = pdf_texts[pdf_name]
    headings = pdf_headings[pdf_name]

    # # split the text into blocks based on the headings
    # text_blocks = split_text_into_blocks(text, headings)
        
    # # clean the text blocks
    # text_blocks = clean_text_blocks(text_blocks)

    # final_prompt = ''
    # for heading, text in text_blocks.items():
    #     final_prompt += heading + ': ' + text + " "

    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt")

    # Chunk the input tensor into smaller parts
    max_chunk_size = 512
    chunked_input_ids = []
    for i in range(0, inputs.input_ids.size(1), max_chunk_size):
        start = i
        end = min(i + max_chunk_size, inputs.input_ids.size(1))
        chunked_input_ids.append(inputs.input_ids[:, start:end])

    # Make predictions for each chunk
    probs_list = []
    with torch.no_grad():
        for chunk in chunked_input_ids:
            outputs = model(input_ids=chunk)
            probs = softmax(outputs.logits, dim=1)
            probs_list.append(probs)

    # Combine the probabilities for all chunks
    combined_probs = torch.cat(probs_list, dim=0)

    # Calculate the average probability for each sentiment class
    average_probs = combined_probs.mean(dim=0)
    positive_sentiment_prob = average_probs[1].item()
    negative_sentiment_prob = average_probs[0].item()

    # Determine the sentiment label based on the probability
    sentiment_label = "Positive" if positive_sentiment_prob > negative_sentiment_prob else "Negative"

    # Store the results
    sentiment_labels[pdf_name] = sentiment_label
    positive_sentiment_probs[pdf_name] = positive_sentiment_prob
    negative_sentiment_probs[pdf_name] = negative_sentiment_prob

    # Print the results
    # print("Input Text:", final_prompt)
    # print("Sentiment Label:", sentiment_label)
    # print("Positive Sentiment Probability:", positive_sentiment_prob)
    # print("Negative Sentiment Probability:", negative_sentiment_prob)

  0%|          | 0/50 [00:00<?, ?it/s]

In [40]:
bart_sentiment_labels_df = pd.DataFrame(list(sentiment_labels.items()), columns=['pdf_name', 'sentiment_label'])
bart_positive_sentiment_probs_df = pd.DataFrame(list(positive_sentiment_probs.items()), columns=['pdf_name', 'positive_sentiment_prob'])
bart_negative_sentiment_probs_df = pd.DataFrame(list(negative_sentiment_probs.items()), columns=['pdf_name', 'negative_sentiment_prob'])

In [41]:
# export csv file
bart_sentiment_labels_df.to_csv("Scores/bart_sentiment_labels.csv", index=False)
bart_positive_sentiment_probs_df.to_csv("Scores/bart_positive_sentiment_probs.csv", index=False)
bart_negative_sentiment_probs_df.to_csv("Scores/bart_negative_sentiment_probs.csv", index=False)

**ROBERTA**

In [33]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from torch.nn.functional import softmax
import torch
import pickle
from tqdm.notebook import tqdm

In [34]:
# Load the pre-trained RoBERTa model and tokenizer
model_name = "roberta-base"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.

In [35]:
roberta_sentiment_labels = {}
roberta_positive_sentiment_probs = {}
roberta_negative_sentiment_probs = {}

for pdf_name in tqdm(pdf_texts):
    # Input text containing Q4 financial results
    # input_text = "We had a beautiful Q4, completing a great year as internet TV expands globally. In 2017, we grew streaming revenue 36% to over $11 billion, added 24 million new memberships (compared to 19 million in 2016), achieved for the first time a full-year positive international contribution profit, and more than doubled global operating income. Average paid streaming memberships rose 25% year over year in Q4. Combined with a 9% increase in ASP, global streaming revenue growth amounted to 35%. Operating income of $245 million (7.5% margin) vs. $154 million prior year (6.2% margin) was slightly above our $238 million forecast. Operating margin for FY17 was 7.2%, on target with our goal at the beginning of this year. EPS was $0.41 vs. $0.15 last year and met our forecast of $0.41. There were several below the line items that affected net income, including a pre-tax $26 million non-cash unrealized loss from F/X remeasurement on our Eurobond. Our tax rate was helped by a $66 million foreign tax benefit, which partially offset a revaluation of our deferred tax assets and the impact from the mandatory deemed repatriation of accumulated foreign earnings related to the recent US tax reform. In Q4, we registered global net adds of 8.3 million, the highest quarter in our history and up 18% vs. last year’s record 7.05 million net adds. This exceeded our 6.3m forecast due primarily to stronger than expected acquisition fueled by our original content slate and the ongoing global adoption of internet entertainment. Geographically, the outperformance vs. guidance was broad-based. In the US, memberships rose by 2.0 million (vs. forecast of 1.25m) bringing total FY17 net adds to 5.3 million. ASP rose 5% year-over-year. Domestic contribution profit increased 5% year-over-year although contribution margin of 34.4% declined both on a year-over-year and sequential basis due to the marketing spend we noted in last quarter’s investor letter. Internationally, we added 6.36 million memberships (compared with guidance of 5.05m), a new record for quarterly net adds for this segment. Excluding a F/X impact of +$43 million, international revenue and ASP grew 59% and 12% year over year, respectively. The increase in ASP reflects price adjustments in a wide variety of our markets over the course of 2017. With contribution profit of $227 million in 2017 (4.5% contribution margin), the international segment delivered its first full year of positive contribution profit in our history. We took a $39m non-cash charge in Q4 for unreleased content we’ve decided not to move forward with. This charge was recognized in content expense in cost of revenues. Despite this unexpected expense, we slightly exceeded our contribution profit and operating income forecast due to our stronger than expected member growth and the timing of international content spend."

    # pdf_name = "FINAL-Q1-19-Shareholder-Letter"

    text = pdf_texts[pdf_name]
    headings = pdf_headings[pdf_name]

    # # split the text into blocks based on the headings
    # text_blocks = split_text_into_blocks(text, headings)
        
    # # clean the text blocks
    # text_blocks = clean_text_blocks(text_blocks)

    # final_prompt = ''
    # for heading, text in text_blocks.items():
    #     final_prompt += heading + ': ' + text + " "

    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt")

    # Chunk the input tensor into smaller parts
    max_chunk_size = 512
    chunked_input_ids = []
    for i in range(0, inputs.input_ids.size(1), max_chunk_size):
        start = i
        end = min(i + max_chunk_size, inputs.input_ids.size(1))
        chunked_input_ids.append(inputs.input_ids[:, start:end])

    # Make predictions for each chunk
    probs_list = []
    with torch.no_grad():
        for chunk in chunked_input_ids:
            outputs = model(input_ids=chunk)
            probs = softmax(outputs.logits, dim=1)
            probs_list.append(probs)

    # Combine the probabilities for all chunks
    combined_probs = torch.cat(probs_list, dim=0)

    # Calculate the average probability for each sentiment class
    average_probs = combined_probs.mean(dim=0)
    positive_sentiment_prob = average_probs[1].item()
    negative_sentiment_prob = average_probs[0].item()

    # Determine the sentiment label based on the probability
    sentiment_label = "Positive" if positive_sentiment_prob > negative_sentiment_prob else "Negative"

    # Store the results
    sentiment_labels[pdf_name] = sentiment_label
    positive_sentiment_probs[pdf_name] = positive_sentiment_prob
    negative_sentiment_probs[pdf_name] = negative_sentiment_prob

    # Print the results
    # print("Input Text:", final_prompt)
    # print("Sentiment Label:", sentiment_label)
    # print("Positive Sentiment Probability:", positive_sentiment_prob)
    # print("Negative Sentiment Probability:", negative_sentiment_prob)

  0%|          | 0/50 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (6136 > 512). Running this sequence through the model will result in indexing errors


In [38]:
roberta_sentiment_labels_df = pd.DataFrame(list(roberta_sentiment_labels.items()), columns=['pdf_name', 'sentiment_label'])
roberta_positive_sentiment_probs_df = pd.DataFrame(list(roberta_positive_sentiment_probs.items()), columns=['pdf_name', 'positive_sentiment_prob'])
roberta_negative_sentiment_probs_df = pd.DataFrame(list(roberta_negative_sentiment_probs.items()), columns=['pdf_name', 'negative_sentiment_prob'])

In [39]:
# export csv file
roberta_sentiment_labels_df.to_csv("Scores/roberta_sentiment_labels.csv", index=False)
roberta_positive_sentiment_probs_df.to_csv("Scores/roberta_positive_sentiment_probs.csv", index=False)
roberta_negative_sentiment_probs_df.to_csv("Scores/roberta_negative_sentiment_probs.csv", index=False)